### Testowanie uczenia się DFA z pomocą systemu przepisywania więzów

In [41]:
import copy, sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../utils")
from importlib import reload

import Inferring, InferringDFA,  InferringconvDFA, utils.automats.convDFA, utils.automats.DFA, utils.automats.PatternDFA
import utils.oracles.oracle1 as oracle1, utils.oracles.oracle1Conv as oracle1Conv
reload(oracle1)
reload(oracle1Conv)
reload(Inferring)
reload(InferringDFA)
reload(InferringconvDFA)
reload(utils.automats.convDFA)
reload(utils.automats.DFA)
reload(utils.automats.PatternDFA)

from Inferring import Inferring
from InferringconvDFA import InferringconvDFA
from utils.automats.convDFA import convDFA
from utils.automats.DFA import DFA
from utils.automats.PatternDFA import PatternDFA
from utils.oracles.oracle1Conv import Oracle1Conv

from InferringDFA import InferringDFA
from utils.oracles.oracle1prof import OracleProf

In [14]:
def run_learning_process(test, oracle=None, check_consistency=False, debug=False):
    _dfa = copy.deepcopy(test.dfa)
    if test.type=="conv":
        # print(f"ucze się języka:")
        # _dfa.print_transitions()
        learn_dfa = InferringconvDFA(_dfa, oracle, check_consistency=check_consistency, debug=debug)
    if test.type=="dfa":
        learn_dfa = InferringDFA(_dfa, oracle, check_consistency=check_consistency, debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    print(f"kontrprzykłady: {cnt_ex}, cnt = {cnt}")
    return copy.deepcopy(dfa) 

In [15]:
class test_example():
    def __init__(self, type, dfa, description=''):
        self.type = type
        self.dfa = copy.deepcopy(dfa)
        self.description = description
        

In [4]:
test_examples = [
    test_example(type = "conv", 
                 dfa = convDFA(type="conv", 
                               dfa1 = DFA(Q=4, input_signs=['a', 'b'], δ = {(0, 'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):2, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):3}, F=set([3])),
                               dfa2 = DFA(Q=4, input_signs=['a', 'b'], δ = {(0, 'a'):0, (0, 'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):0, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3]))),
                 description = "splot, dfa1 - wzorzec 'aba', dfa2 - wzorzec 'bbb'"),
    
    test_example(type="conv", 
                 dfa = convDFA(type="conv", 
                               dfa1=DFA(Q=3, input_signs=['a', 'b'], δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):2, (2, 'b'):2}, F= set([2])), 
                               dfa2=DFA(Q=3, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):2, (2, 'b'):2}, F= set([2]))),
                description="splot, dfa1 - wzorzec - 'bb', dfa2 - wzorzec 'aa'" ),
    
    test_example(type="conv", 
                 dfa = convDFA(type="conv", 
                               dfa1=DFA(Q=4, input_signs=['a', 'b'], δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):0, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F= set([3])),
                               dfa2=DFA(Q=4, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):3}, F= set([3]))),
                 description="splot, dfa1 - wzorzec 'bbb', dfa2 - wzorzec 'aaa'"),
                 
    test_example(type="conv",
                 dfa=convDFA(type="conv",
                             dfa1=DFA(Q=2, input_signs=['a', 'b'], δ = {(0,'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):1}, F=set([1])),
                             dfa2=DFA(Q=2, input_signs=['a', 'b'], δ = {(0,'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):1}, F=set([1]))),
                description="splot, dfa1 - słowa zawierające literke 'a', dfa2 - słowa zawierające literke 'a'"),
                
    test_example(type="conv", 
                 dfa=convDFA(type="conv",
                             dfa1=DFA(Q=4, input_signs=['a', 'b'], δ={(0, 'a'):2, (0,'b'):1, (1, 'b'):1, (1, 'a'):3, (2, 'a'):2, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3])),
                             dfa2=DFA(Q=4, input_signs=['a', 'b'], δ={(0, 'a'):2, (0,'b'):1, (1, 'b'):1, (1, 'a'):3, (2, 'a'):2, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3]))),
                description="splot, dfa1 - słowa zawierające i literę 'a' i literę 'b', dfa2 - słowa zawierające i literę 'a' i literę 'b'"),
                
    test_example(type="dfa", 
                 dfa=DFA(Q=3, input_signs=['0', '1', '2', '3', '4', '5', '6', '7'], δ={(0,'0'):0, (0,'1'):2, (0,'2'):2, (0,'3'):0, (0,'4'):2, (0,'5'):0, (0,'6'):1, (0,'7'):2, (1,'0'):2, (1,'1'):0, (1,'2'):1, (1,'3'):2, (1,'4'):1, (1,'5'):2, (1,'6'):2, (1,'7'):1, (2,'0'):2, (2,'1'):2, (2,'2'):2, (2,'3'):2, (2,'4'):2, (2,'5'):2, (2,'6'):2, (2,'7'):2}, F=set([0])),
                 description="zakodowane dodawania w systemie dwójkowym"), 
    
    test_example(type="conv", 
                 dfa=convDFA(type="conv", 
                             dfa1=DFA(Q=5, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):2, (3,'a'):1, (3,'b'):4, (4,'a'):4, (4,'b'):4}, F=set([2, 4])),
                             dfa2=DFA(Q=5, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):2, (3,'a'):1, (3,'b'):4, (4,'a'):4, (4,'b'):4}, F=set([2, 4])),
                description="splot, dfa1 - 2 wzorce 'aa' lub 'bb', dfa2 - 2 wzorce 'aa' lub 'bb'")),
                
    test_example(type="conv", 
                 dfa=convDFA(type="conv", 
                             dfa1=DFA(Q=9, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):5, (3,'a'):1, (3,'b'):4, (4,'a'):7, (4,'b'):4, (5,'a'):5, (5,'b'):6, (6,'a'):6, (6,'b'):6, (7,'a'):8, (7,'b'):7, (8,'a'):8, (8,'b'):8}, F=set([6, 8])),
                             dfa2=DFA(Q=9, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):5, (3,'a'):1, (3,'b'):4, (4,'a'):7, (4,'b'):4, (5,'a'):5, (5,'b'):6, (6,'a'):6, (6,'b'):6, (7,'a'):8, (7,'b'):7, (8,'a'):8, (8,'b'):8}, F=set([6, 8]))),
                description="splot, dfa1 - 2 wzorce 'aa' i 'bb', dfa2 - 2 wzorce 'aa' i 'bb'")]

"""splot, dfa1 - 2 wzorce 'aa' i 'bb', dfa2 - 2 wzorce 'aa' i 'bb'
kontrprzykłady: [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,  8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8], cnt = [10872, 39]
kontrprzykłady: [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,  8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8], cnt = [4547, 39]
kontrprzykłady: [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 8, 8, 8, 8, 9, 8, 9, 8, 8, 8, 8, 8, 8], cnt = [3485, 4]"""

"splot, dfa1 - 2 wzorce 'aa' i 'bb', dfa2 - 2 wzorce 'aa' i 'bb'\nkontrprzykłady: [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,  8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8], cnt = [10872, 39]\nkontrprzykłady: [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,  8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8], cnt = [4547, 39]\nkontrprzykłady: [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 8, 8, 8, 8, 9, 8, 9, 8, 8, 8, 8, 8, 8], cnt = [3485, 4]"

In [38]:
dfa1 = PatternDFA(input_signs=['a', 'b'], patterns=['aa', 'bb'])
dfa2 = PatternDFA(input_signs=['a', 'b', 'c'], patterns=['aa', 'bb', 'c'])

q = dfa1.δ[(4,'a')]
print(f"q = {q}")
# print(f"Q1 = {dfa1.Q}, Q2 = {dfa2.Q}")
# dfa=convDFA(type="conv", dfa1=dfa1, dfa2=dfa2)

q = 6


In [43]:
test = test_example(type="conv",
                    dfa=convDFA(type="conv",
                                dfa1=PatternDFA(input_signs=['a', 'b'], patterns=['aa', 'bb']),
                                dfa2=PatternDFA(input_signs=['a', 'b', 'c'], patterns=['aa', 'bb', 'c'])),
                    description="splot, dfa1 - 2 wzorce: 'aa', 'bb', dfa2 - 3 wzorce: 'aa', 'bb', 'c'")

print(f"{test.description}")
d = run_learning_process(copy.deepcopy(test), oracle=None, check_consistency=False, debug=False)

splot, dfa1 - 2 wzorce: 'aa', 'bb', dfa2 - 3 wzorce: 'aa', 'bb', 'c'
kontrprzykłady: [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9], cnt = [46811, 66]


In [44]:
d = run_learning_process(copy.deepcopy(test), oracle=Oracle1Conv(), check_consistency=False, debug=False)

KeyboardInterrupt: 

In [ ]:
d = run_learning_process(copy.deepcopy(test), oracle=Oracle1Conv(), check_consistency=True, debug=False)

In [6]:
test = test_examples[1]
print(f"{test.description}")

d = run_learning_process(copy.deepcopy(test), oracle=None, check_consistency=False, debug=False)
d = run_learning_process(copy.deepcopy(test), oracle=Oracle1Conv(), check_consistency=False, debug=False)
d = run_learning_process(copy.deepcopy(test), oracle=Oracle1Conv(), check_consistency=True, debug=False)

splot, dfa1 - wzorzec - 'bb', dfa2 - wzorzec 'aa'
kontrprzykłady: [4, 4, 4, 4], cnt = [289, 5]
kontrprzykłady: [4, 4, 4, 4], cnt = [150, 5]
kontrprzykłady: [4, 4, 4, 8], cnt = [170, 2]


In [155]:
test = test_example(type="dfa", dfa=DFA(Q=9, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):5, (3,'a'):1, (3,'b'):4, (4,'a'):7, (4,'b'):4, (5,'a'):5, (5,'b'):6, (6,'a'):6, (6,'b'):6, (7,'a'):8, (7,'b'):7, (8,'a'):8, (8,'b'):8}, F=set([6, 8])))
d = run_learning_process(copy.deepcopy(test), oracle=None, check_consistency=False, debug=False)
d.print_transitions()

kontrprzykłady: [4, 4], cnt = [71, 3]
(0,a) --> 6
(0,b) --> 1
(1,a) --> 6
(1,b) --> 2
(2,a) --> 4
(2,b) --> 2
(3,a) --> 3
(3,b) --> 3
(4,a) --> 3
(4,b) --> 4
(5,a) --> 5
(5,b) --> 3
(6,a) --> 7
(6,b) --> 1
(7,a) --> 7
(7,b) --> 5
stany akceptujące - {3}


### Uczenie się splotu DFA - **debugowanie** 

<!-- ### Uczenie się splotów - **debugowanie**  -->

In [66]:
import copy, sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../utils")
from importlib import reload

import Inferring, InferringDFA,  InferringconvDFA, utils.automats.convDFA, utils.automats.DFA
import utils.oracles.oracle1 as oracle1, utils.oracles.oracle1Conv as oracle1Conv
reload(oracle1)
reload(oracle1Conv)
reload(Inferring)
reload(InferringDFA)
reload(InferringconvDFA)
reload(utils.automats.convDFA)
reload(utils.automats.DFA)

from Inferring import Inferring
from InferringconvDFA import InferringconvDFA
from utils.automats.convDFA import convDFA
from utils.automats.DFA import DFA
from utils.oracles.oracle1Conv import Oracle1Conv

def run_learning_process(_dfa, oracle=None, debug=False, check_consistency=False):
    learn_dfa = InferringconvDFA(_dfa, oracle, debug=debug, check_consistency=check_consistency)
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    print(f"kontrprzykłady: {cnt_ex}, cnt = {cnt}")
    return copy.deepcopy(dfa) 

In [69]:
"""wzorzec - bb"""
dfa1 = DFA(Q=3, input_signs=['a', 'b'], 
           δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):2, (2, 'b'):2}, F= set([2]))
"""wzorzec - aa"""
dfa2 = DFA(Q=3, input_signs=['a', 'b'], 
           δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):2, (2, 'b'):2}, F= set([2]))

convdfa = convDFA(type='conv', dfa1=dfa1, dfa2=dfa2)
d = run_learning_process(copy.deepcopy(convdfa), oracle=Oracle1Conv(), debug=False, check_consistency=True)

kontrprzykłady: [4, 4, 4, 4], cnt = [153, 2]
